<a href="https://colab.research.google.com/github/lalesafarzade/Recommendation_system_Project/blob/lale/Notebooks/2.Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 256 kB in 2s (109 kB/s)
Reading 

In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


In [35]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movie").config("spark.executor.memory", "1G").config("spark.executor.cores","4").getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
url = '/content/drive/MyDrive/movies.csv'

movie = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

movie.cache()
movie.show(2)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows



In [7]:
url = '/content/drive/MyDrive/ratings.csv'

rating = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

rating.cache() #for faster re-use
rating.show(2)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
+------+-------+------+----------+
only showing top 2 rows



In [8]:
rating.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:
#df = rating.join(movie,['movieId'],how='inner').distinct()
#df.show(5)

In [11]:
# Find Count of Null, None, NaN of All DataFrame Columns
#from pyspark.sql.functions import col,isnan, when, count
#df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [12]:
(training, test) = rating.randomSplit([0.8, 0.2])

In [14]:
als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop", nonnegative = True, implicitPrefs = False,rank=50)

In [15]:
model = als.fit(training)

In [16]:
predictions = model.transform(test)

In [17]:
predictions.show(n = 10)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   165|     31|   5.0| 846330065| 3.7113938|
|   230|     31|   3.0| 862573039|  3.152259|
|   597|     31|   2.0|1018202809| 3.1153636|
|   613|     31|   4.0|1111475597| 3.2259483|
|   631|     31|   3.0| 833454590| 3.5143418|
|   858|     31|   3.0| 973575996| 2.4985316|
|   992|     31|   4.0| 847061924| 3.0209541|
|  1260|     31|   3.0| 833419568|  3.030496|
|  1624|     31|   4.0| 831055670|  3.425091|
|  1645|     31|   4.0| 848985672| 3.1393178|
+------+-------+------+----------+----------+
only showing top 10 rows



In [18]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8112601974550638


In [19]:
import tempfile
path = tempfile.mkdtemp()
model_path = path + "/spark_model"
model.write().save(model_path)

In [20]:
model_path

'/tmp/tmp9ygy_a4u/spark_model'

In [22]:
path = F"/content/drive/MyDrive/spark_model"
model.write().save(path)

In [30]:
loaded_model=model.read().load(path)

In [24]:
pred = loaded_model.transform(test)

In [25]:
predictions.show(n = 20)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   165|     31|   5.0| 846330065| 3.7113938|
|   230|     31|   3.0| 862573039|  3.152259|
|   597|     31|   2.0|1018202809| 3.1153636|
|   613|     31|   4.0|1111475597| 3.2259483|
|   631|     31|   3.0| 833454590| 3.5143418|
|   858|     31|   3.0| 973575996| 2.4985316|
|   992|     31|   4.0| 847061924| 3.0209541|
|  1260|     31|   3.0| 833419568|  3.030496|
|  1624|     31|   4.0| 831055670|  3.425091|
|  1645|     31|   4.0| 848985672| 3.1393178|
|  1659|     31|   4.0| 836493978|  3.369099|
|  1894|     31|   2.0| 974801944| 2.6661594|
|  1946|     31|   3.5|1165650805| 3.5106306|
|  2469|     31|   3.0|1081743980| 3.2750523|
|  2886|     31|   3.0| 834335076| 3.0280285|
|  3517|     31|   4.0|1109532345| 3.0056381|
|  3703|     31|   3.0| 847570948| 3.2309895|
|  3757|     31|   2.0|1112328270| 2.7186775|
|  3970|     31|   4.0|1232065220|

In [26]:
#Generate top 5 movie recommendation for each user
userRecommendation=model.recommendForAllUsers(5)

/content/spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [27]:
userRecommendation.select("userId","recommendations.movieId").show(10,False)

+------+----------------------------------------+
|userId|movieId                                 |
+------+----------------------------------------+
|1     |[194334, 151989, 192089, 155923, 183947]|
|3     |[203086, 194434, 180323, 151989, 203882]|
|5     |[203086, 194434, 183947, 151989, 203882]|
|6     |[194434, 151989, 203086, 183947, 203882]|
|9     |[179985, 203086, 194434, 180323, 166812]|
|12    |[194434, 151989, 183947, 203086, 194334]|
|13    |[194434, 202231, 203086, 183947, 151989]|
|15    |[203086, 183947, 203882, 151989, 194434]|
|16    |[203086, 194434, 151989, 183947, 203882]|
|17    |[199187, 180323, 175625, 176597, 203086]|
+------+----------------------------------------+
only showing top 10 rows



In [61]:
specific_user_preds = spark.createDataFrame([(3, 7000),(4, 7000),(7, 7000)], [ "userId","movieId"])
specific_user_preds.show()


+------+-------+
|userId|movieId|
+------+-------+
|     3|   7000|
|     4|   7000|
|     7|   7000|
+------+-------+



In [56]:
#prediction some movie for a specific user
new_predictions=loaded_model.transform(specific_user_preds)
new_predictions.show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|     3|   7000|  3.123271|
|     4|   7000| 2.5981631|
|     7|   7000| 2.5006652|
+------+-------+----------+

